In [ ]:
import  openpyxl 
import pandas as pd
import numpy as np
import os
import re


def process_excel_files(folder_path, file_names, sheet_name):
    summed_data = {}        # Dictionary to store summed data for each cell.
    files_with_sheet = 0       # Counter for files containing the specified sheet.

    for file_name in file_names:
        #File opening
        file_path = os.path.join(folder_path, file_name)
        workbook = openpyxl.load_workbook(file_path, data_only=True)
        
        #Checking version of PIT
        cell_value = workbook['Info'].cell(row=4, column=4).value
        match = re.search(r"Version:\s*([\d.]+)\s*as", cell_value or "")
        version = float(match.group(1)[:3]) if match else None
        
        # Skip the file if the version is not found or is less than 2.5 or if the sheet is not present.
        if not version or version < 2.5 or sheet_name not in workbook.sheetnames:
            continue
        
        files_with_sheet += 1  
        sheet = workbook[sheet_name]

        #Extract values from the specified cell range C1:D86 and store them in a list of dictionaries.
        cell_range = sheet['C1:D86'] 
        data = [{'Cell': cell.coordinate,'Count_values': 1 if cell.value is not None else 0}
                for row in cell_range for cell in row]
        cell = sheet['F2']
        data.append({'Cell': cell.coordinate,'Count_values': 1 if cell.value is not None else 0})
        #print(data)

        for entry in data:
            cell_coord = entry['Cell']
            if cell_coord not in summed_data:
                summed_data[cell_coord] = {'Count_values': 0}
            summed_data[cell_coord]['Count_values'] += entry['Count_values']
            
    # Calculate frequency for each cell and store it in result_data.
    result_data = {}
    for cell, values in summed_data.items():
        frequency = values['Count_values'] / files_with_sheet
        result_data[cell] = round(frequency, 2)

    return result_data

In [ ]:
folder_path = os.path.join(os.getcwd(), "Audi_SSP_LK3 _4")
sheet_name = 'CAT - Overview'
file_names = [f for f in os.listdir(folder_path)]
result_data = process_excel_files(folder_path, file_names, sheet_name)

c:\Users\iwachpul\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\reader\drawings.py:67: UserWarning: wmf image format is not supported so the image is being dropped
  warn(msg)
c:\Users\iwachpul\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


[{'Cell': 'C1', 'Count_values': 0}, {'Cell': 'D1', 'Count_values': 0}, {'Cell': 'C2', 'Count_values': 1}, {'Cell': 'D2', 'Count_values': 0}, {'Cell': 'C3', 'Count_values': 1}, {'Cell': 'D3', 'Count_values': 0}, {'Cell': 'C4', 'Count_values': 0}, {'Cell': 'D4', 'Count_values': 0}, {'Cell': 'C5', 'Count_values': 0}, {'Cell': 'D5', 'Count_values': 0}, {'Cell': 'C6', 'Count_values': 0}, {'Cell': 'D6', 'Count_values': 0}, {'Cell': 'C7', 'Count_values': 0}, {'Cell': 'D7', 'Count_values': 0}, {'Cell': 'C8', 'Count_values': 0}, {'Cell': 'D8', 'Count_values': 0}, {'Cell': 'C9', 'Count_values': 0}, {'Cell': 'D9', 'Count_values': 0}, {'Cell': 'C10', 'Count_values': 0}, {'Cell': 'D10', 'Count_values': 0}, {'Cell': 'C11', 'Count_values': 0}, {'Cell': 'D11', 'Count_values': 0}, {'Cell': 'C12', 'Count_values': 1}, {'Cell': 'D12', 'Count_values': 0}, {'Cell': 'C13', 'Count_values': 0}, {'Cell': 'D13', 'Count_values': 0}, {'Cell': 'C14', 'Count_values': 1}, {'Cell': 'D14', 'Count_values': 0}, {'Cell': 

In [ ]:
result_sheet = openpyxl.load_workbook('My_version_template.xlsx', keep_vba= False)
ws = result_sheet['CAT - Overview'] 

for key, value in result_data.items():
    ws[key] = value

result_sheet.save('PIT_Template_v.2.6_edytowany.xlsx')